<a href="https://colab.research.google.com/github/hawa1983/Capstone/blob/main/ibx_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NYC Subway Station Catchment Population Analysis
Goal: Calculate the population within a 0.5-mile radius (catchment area) of each New York City subway station using 2020 census block group data. We will use Python libraries (Pandas, GeoPandas, Shapely) and the U.S. Census API to gather required data and perform spatial analysis. The steps include:
- Loading and cleaning the GTFS stops.txt to get subway station locations.
- Downloading Census TIGER/Line shapefiles for 2020 block group boundaries in NYC (the five boroughs).
- Using the Census API (ACS 5-Year 2020, variable B01003_001) to get total population for each block group.
- Merging population data with the block group geometries.
- Creating a 0.5-mile buffer around each station.
- Intersecting each station’s buffer with block group polygons and using area-weighted interpolation to estimate how much of each block group’s population lies within the buffer
walker-data.com
.
- Summing the weighted populations to get the total catchment population for each station, and outputting the results to a CSV.
## 1. Load and Prepare Subway Stops Data
First, load the GTFS stops.txt file into a Pandas DataFrame and filter it to get only subway stations (not individual entrances or platforms). In GTFS, subway stations are typically marked with location_type = 1 (parent stations), while individual stop platforms have location_type = 0 and a parent_station reference. We will keep only records where location_type = 1 to represent each station once. Then we convert the DataFrame to a GeoDataFrame using latitude and longitude coordinates.

In [36]:
import pandas as pd
import geopandas as gpd
from io import BytesIO
from zipfile import ZipFile
import requests

# Download the GTFS ZIP from the MTA URL
url = "http://web.mta.info/developers/data/nyct/subway/google_transit.zip"
response = requests.get(url)
with ZipFile(BytesIO(response.content)) as z:
    # Read stops.txt directly from inside the ZIP
    with z.open("stops.txt") as f:
        stops_df = pd.read_csv(f)

# Filter for station-level entries
stations_df = stops_df[stops_df['location_type'] == 1].copy()

# Convert to GeoDataFrame
stations_gdf = gpd.GeoDataFrame(
    stations_df,
    geometry=gpd.points_from_xy(stations_df.stop_lon, stations_df.stop_lat),
    crs="EPSG:4326"
)

print("Number of stations loaded:", len(stations_gdf))
print(stations_gdf.head(3))


Number of stations loaded: 496
  stop_id                  stop_name  stop_lat  stop_lon  location_type  \
0     101  Van Cortlandt Park-242 St        41       -74              1   
3     103                     238 St        41       -74              1   
6     104                     231 St        41       -74              1   

  parent_station                    geometry  
0            NaN  POINT (-73.89858 40.88925)  
3            NaN  POINT (-73.90087 40.88467)  
6            NaN  POINT (-73.90483 40.87886)  


## 2. Retrieve Census Block Group Geometries for NYC

To determine populations around stations, we need the geographic boundaries of census block groups in NYC. Block groups are small areas used by the Census (each block group typically has 600–3,000 people
catalog.data.gov, making them fine-grained units for population data). We will use the TIGER/Line shapefiles for 2020 block groups provided by the U.S. Census Bureau catalog.data.gov.

We'll download the shapefile for all block groups in New York State and then filter it to the five boroughs (New York County/Manhattan, Bronx, Kings/Brooklyn, Queens, Richmond/Staten Island). Each of these boroughs corresponds to a county with FIPS codes 061, 005, 047, 081, 085 respectively (when prefixed with state FIPS 36 for New York). After loading the shapefile with GeoPandas, we filter by the county codes.

**Explanation:** We use the Census TIGER shapefile for 2020 block groups in New York. GeoPandas can read directly from a zip URL. The shapefile’s attribute table has fields like STATEFP20, COUNTYFP20, TRACTCE20, BLKGRPCE20, and GEOID20 (or similar), representing state, county, tract, block group codes and a concatenated GEOID. We filter by COUNTYFP20 values to get only the block groups in NYC’s five counties.

**Note:** The CRS of the loaded shapefile is likely a geographic coordinate system (NAD83). Before doing area calculations or buffering, we will need to project these geometries to a planar coordinate system with units in meters (discussed in a later step).

In [37]:
# Corrected version
shapefile_url = "https://www2.census.gov/geo/tiger/TIGER2020/BG/tl_2020_36_bg.zip"
block_groups_gdf = gpd.read_file(shapefile_url)

# Filter for NYC’s five counties (boroughs)
nyc_county_fips = ["005", "047", "061", "081", "085"]  # Bronx, Brooklyn, Manhattan, Queens, Staten Island
block_groups_nyc = block_groups_gdf[block_groups_gdf["COUNTYFP"].isin(nyc_county_fips)].copy()

print("Total NYC block groups:", len(block_groups_nyc))
block_groups_nyc.head(2)


Total NYC block groups: 6807


,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
24,36,061,023900,1,360610239001,Block Group 1,G5030,S,27517,0,+40.8322236,-073.9404112,"POLYGON ((-73.94112 40.83166, -73.94088 40.832..."
25,36,061,013900,1,360610139001,Block Group 1,G5030,S,23621,0,+40.7688543,-073.9868884,"POLYGON ((-73.98806 40.76979, -73.98666 40.769..."


## 3. Retrieve Population Data for Block Groups (ACS 5-Year 2020)

Next, we gather population data for each block group. We use the American Community Survey (ACS) 5-Year 2020 data, specifically the variable B01003_001 which is the total population of the block group. We can query the Census API for this data. The API requires a valid Census API key (you can obtain one for free from the Census Bureau). Insert your API key in the code where indicated.

We'll query all block groups in the five NYC counties. The API call will specify the state (36 for New York) and each county FIPS. We can either make one call per county or attempt a single call with multiple counties. For simplicity, we'll loop over the counties.

**Explanation:** We call the ACS API for each county. The query requests the field B01003_001E (total population estimate) for all block groups in the specified state and county (for=block group:*&in=state:36&in=county:XYZ). The response is JSON with each row containing the population and geographic identifiers (state, county, tract, block group). We combine the results and create a GEOID that matches the one in the shapefile (concatenating state, county, tract, block group codes). We also convert the population to numeric type.

**Note:** Ensure you have a valid Census API key in place of "YOUR_CENSUS_API_KEY". Without a key, the API might rate-limit or reject the request. If you prefer, you could use the cenpy library to retrieve ACS data as well, but here we use direct requests for transparency.

In [38]:
import requests

# Your Census API key
API_KEY = "a4373ffe644694a60eeb7c0bae0bedcfd2d6ff78"

# Prepare a DataFrame to collect population data
pop_list = []
for county in nyc_county_fips:
    url = (
        "https://api.census.gov/data/2020/acs/acs5?get=B01003_001E"
        f"&for=block%20group:*&in=state:36&in=county:{county}&key={API_KEY}"
    )
    response = requests.get(url)
    data = response.json()
    # The first row of the response is the header
    columns = data[0]
    values = data[1:]
    df = pd.DataFrame(values, columns=columns)
    pop_list.append(df)

# Concatenate all counties data
pop_df = pd.concat(pop_list, ignore_index=True)
pop_df.rename(columns={"B01003_001E": "population"}, inplace=True)
# Convert population to numeric
pop_df["population"] = pd.to_numeric(pop_df["population"])
# Construct the full 12-digit GEOID for block group (state+county+tract+block group codes)
pop_df["GEOID"] = pop_df["state"] + pop_df["county"] + pop_df["tract"] + pop_df["block group"]
pop_df.head(3)


,population,state,county,tract,block group,GEOID
0,6600,36,005,000100,1,360050001001
1,1542,36,005,000200,2,360050002002
2,2830,36,005,002001,1,360050020011


## 4. Merge Population Data with Block Group Geometries

Now we join the population data with the GeoDataFrame of NYC block group geometries. This will give each block group polygon a population attribute. The join can be done on the GEOID field (the shapefile might have a field named GEOID or GEOID20). We'll ensure our DataFrame’s GEOID string matches the shapefile’s GEOID format.

**Explanation:** We perform a left join so that each block group polygon gets a population value from ACS. All NYC block groups should find a match in the population DataFrame. After this, block_groups_nyc contains geometry and population for each block group. We are now ready to perform spatial analysis.

In [39]:
# Ensure the GEOID field in block_groups_nyc matches the format (could be GEOID or GEOID20)
geo_field = "GEOID" if "GEOID" in block_groups_nyc.columns else "GEOID20"
block_groups_nyc[geo_field] = block_groups_nyc[geo_field].astype(str)

# Merge population into block group geodataframe
block_groups_nyc = block_groups_nyc.merge(pop_df[["GEOID", "population"]],
                                          left_on=geo_field, right_on="GEOID",
                                          how="left")
print("Columns in block_groups_nyc:", block_groups_nyc.columns.tolist())
block_groups_nyc.head(2)


Columns in block_groups_nyc: ['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry', 'population']


,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,population
0,36,061,023900,1,360610239001,Block Group 1,G5030,S,27517,0,+40.8322236,-073.9404112,"POLYGON ((-73.94112 40.83166, -73.94088 40.832...",1538
1,36,061,013900,1,360610139001,Block Group 1,G5030,S,23621,0,+40.7688543,-073.9868884,"POLYGON ((-73.98806 40.76979, -73.98666 40.769...",2059


## 5. Buffer Each Station by 0.5 Miles

Each subway station is considered the center of a catchment area with a 0.5-mile radius. To construct this, we create a circular buffer of 0.5 miles around each station point. Important: We must project our data to a planar coordinate system (with distance in consistent units) before buffering. Working in latitude/longitude (degrees) would give incorrect distances and areas
geopandas.org
. We'll use a projection in feet or meters appropriate for NYC (for example, EPSG:32618 is UTM zone 18N which covers New York in meters, or EPSG:2263 which is NY State Plane in feet). Here, we'll use a meter-based CRS so the buffer distance can be given in meters.

We will transform both the station points and block group polygons to the same projected CRS.

**Explanation:** We use GeoDataFrame.to_crs() to convert geographies to UTM Zone 18N which uses meters. The buffer distance is calculated as 0.5 * 1609.34 (since 1 mile is ~1609.34 meters). Using GeoSeries.buffer() generates a polygon around each point with the given radius. Now stations_proj has an extra column buffer_geom containing the 0.5-mile radius polygon for each station.

In [40]:
# Project both stations and block groups to a planar CRS for accurate distance/area calculations
projected_crs = "EPSG:32618"  # WGS 84 / UTM Zone 18N (meters)
stations_proj = stations_gdf.to_crs(projected_crs)
block_groups_proj = block_groups_nyc.to_crs(projected_crs)

# Add a buffer geometry of 0.5 miles around each station (0.5 mile ≈ 804.67 meters)
buffer_distance = 0.5 * 1609.34  # miles to meters conversion
stations_proj["buffer_geom"] = stations_proj.geometry.buffer(buffer_distance)

stations_proj.head(2)[["stop_id", "stop_name", "buffer_geom"]]


,stop_id,stop_name,buffer_geom
0,101,Van Cortlandt Park-242 St,"POLYGON ((593591.236 4527046.49, 593587.361 45..."
3,103,238 St,"POLYGON ((593404.955 4526535.53, 593401.081 45..."


## 6. Intersect Buffers with Block Groups and Calculate Area-Weighted Population

For each station’s buffer, we find which block group polygons it overlaps and how much of each. The population within the buffer can be estimated by assuming population is uniformly distributed across each block group’s area. Then the fraction of a block group’s area that falls inside the buffer is the fraction of that block group’s population we count towards the station. This is the area-weighted interpolation approach
walker-data.com
 – using area of overlap as weights to allocate population from polygons to the buffer region.

We will iterate over each station buffer, intersect it with the block group geometries, compute area overlaps, and sum the weighted populations:

Explanation: We iterate through each station’s buffer polygon. Using a spatial index speeds up the search by first filtering block groups whose bounding box intersects the buffer’s bounds. Then we filter to actual intersections. For each intersecting block group, we calculate the polygon of overlap (intersection_area) and divide by the block group’s total area to get area_weight. We multiply this weight by the block group’s population to get the portion of the population inside the buffer. Finally, summing these contributions yields the catchment population for the station.

This method assumes population is uniformly distributed within a block group, which is a common assumption for area-weighted interpolation
walker-data.com
. Note that block groups are small (hundreds to a few thousand people
catalog.data.gov
), so this assumption is reasonably accurate at the city scale, though it could over/underestimate in some cases (e.g., if a buffer cuts through a block group with uneven population distribution).

We also included each station’s stop_id, stop_name, and original latitude/longitude (if needed, we could get lat/lon from the original data before projection to avoid any numeric changes due to projection). The catchment population is rounded to one decimal place in this example; you may keep it as an integer if desired.

In [41]:
from shapely.geometry import Polygon

# Precompute each block group's full area (in projected CRS units, e.g., square meters)
block_groups_proj["area"] = block_groups_proj.geometry.area

catchment_results = []  # list to collect population results for each station

# Use spatial index for efficiency to find candidate block groups for each buffer
bg_sindex = block_groups_proj.sindex

for idx, station in stations_proj.iterrows():
    station_id = station["stop_id"]
    station_name = station["stop_name"]
    buffer_poly = station["buffer_geom"]

    # Find block groups whose bounding box intersects the buffer's bounding box (spatial index pre-filter)
    possible_matches_index = list(bg_sindex.intersection(buffer_poly.bounds))
    possible_bgs = block_groups_proj.iloc[possible_matches_index]
    # Refine: take only those that actually intersect the buffer geometry
    intersecting_bgs = possible_bgs[possible_bgs.intersects(buffer_poly)].copy()
    if intersecting_bgs.empty:
        catchment_pop = 0
    else:
        # Compute intersection polygon area for each overlapping block group
        intersecting_bgs["intersection_area"] = intersecting_bgs.geometry.intersection(buffer_poly).area
        # Calculate area weight (fraction of block group area within buffer)
        intersecting_bgs["area_weight"] = intersecting_bgs["intersection_area"] / intersecting_bgs["area"]
        # Estimate population in buffer = area weight * total block group population
        intersecting_bgs["pop_in_buffer"] = intersecting_bgs["area_weight"] * intersecting_bgs["population"]
        # Sum up the population contributions from all intersecting block groups
        catchment_pop = intersecting_bgs["pop_in_buffer"].sum()

    catchment_results.append({
        "stop_id": station_id,
        "stop_name": station_name,
        "lat": station.geometry.y,   # Note: station.geometry is projected; use original if needed
        "lon": station.geometry.x,
        "catchment_population": round(catchment_pop, 1)  # rounding to 1 decimal (optional)
    })

# Convert results to DataFrame
catchment_df = pd.DataFrame(catchment_results)
catchment_df.head(5)


,stop_id,stop_name,lat,lon,catchment_population
0,101,Van Cortlandt Park-242 St,"4,527,046","592,787","16,760"
1,103,238 St,"4,526,536","592,600","30,776"
2,104,231 St,"4,525,886","592,274","43,778"
3,106,Marble Hill-225 St,"4,525,404","591,859","38,729"
4,107,215 St,"4,524,830","591,407","29,387"


## 7. Output Results to CSV

Finally, we save the results to a CSV file with the requested columns: stop_id, stop_name, lat, lon, catchment_population. This CSV can be used for further analysis or visualization.

The CSV file station_catchment_population.csv will contain one row per station, for example:

Each station’s catchment_population is the sum of area-weighted block group population within 0.5 miles of that station. This completes the computation of subway station catchment populations. You can further utilize this data for analysis such as ranking stations by surrounding population or mapping the results.

Sources:

NYC subway station locations from GTFS stops.txt (user-provided data).

U.S. Census Bureau TIGER/Line Shapefiles for 2020 Block Groups
catalog.data.gov
.

American Community Survey 5-Year 2020 data for block group populations (variable B01003_001).

Methodology of area-weighted interpolation for spatial data
walker-data.com
geopandas.org
 (assuming uniform population distribution over block group areas
walker-data.com
).

Definition and typical size of census Block Groups
catalog.data.gov
.

In [42]:
# Save the results to CSV
catchment_df.to_csv("station_catchment_population.csv", index=False)

print("Saved station_catchment_population.csv with columns:", catchment_df.columns.tolist())


Saved station_catchment_population.csv with columns: ['stop_id', 'stop_name', 'lat', 'lon', 'catchment_population']


# Jobs Within 0.5 Miles of NYC Subway Stations (LODES 2021 and Census Block Groups)

**Introduction**

This analysis computes the number of jobs within a 0.5-mile radius of each New York City subway station. We use the LODES Workplace Area Characteristics (WAC) data (2021, New York) for employment counts, and the 2020 Census block group boundaries for spatial analysis. The methodology mirrors a previous population analysis by using area-weighted interpolation – i.e. allocating job counts from block group polygons to station buffer areas proportional to the area of overlap
gis.stackexchange.com
. We produce a CSV listing each station’s total jobs and jobs in key industry “supersectors” (e.g. retail, education, health, manufacturing). All data used are open-source: the U.S. Census LEHD program for LODES employment
census.gov
, TIGER/Line for block group shapefiles, and the MTA’s GTFS feed for station locations.

**Data sources:**

- LODES WAC 2021 (NY): Provides job counts by workplace location at the Census block level, including total employment and breakdowns by industry sector. The WAC file for New York 2021 (LODES version 8.1) contains a record for each census block’s jobs, with fields such as C000 (total jobs) and CNSxx for NAICS sector categories
lehd.ces.census.gov
lehd.ces.census.gov
. For example, CNS07 = Retail Trade (NAICS 44-45), CNS15 = Educational Services (NAICS 61), CNS16 = Health Care and Social Assistance (NAICS 62), and CNS05 = Manufacturing (NAICS 31-33)
lehd.ces.census.gov
lehd.ces.census.gov
. We will use these fields for our supersector job counts.

- 2020 Block Group Shapefiles (NYC): We obtain the TIGER/Line 2020 shapefile for Census block groups in New York State and filter to the five NYC counties (Bronx, Kings, New York, Queens, Richmond)
en.wikipedia.org
. Block groups are subdivisions of census tracts (generally 600–3,000 people) and share the first digit of their 4-digit census block codes
catalog.data.gov
. Block group “0” codes represent water-only areas with no population
catalog.data.gov
 (and presumably no jobs), which we will exclude.

- GTFS stops.txt (NYC Subway): Provided by the user, containing subway stop locations. We will filter for records with location_type = 1, which correspond to station coordinates (as opposed to individual stop platforms)
gtfs.org
. These station points will serve as the centers of our 0.5-mile buffers.

The analysis proceeds through data download/preparation, constructing station buffers, spatially joining job data to geography, performing area-weighted interpolation, and aggregating results by station.

## 1. Download Latest LODES WAC Data for NY (2021)

First, we download the Workplace Area Characteristics (WAC) file for New York 2021 from the Census LEHD site
census.gov
- According to Census, the LODES 8.1 release includes 2021 data for most states
census.gov
- The file is a CSV (gzipped) with records by workplace census block. We will use Python’s requests to fetch the file and pandas to load the data.

Important: We limit the columns to those needed for efficiency. Specifically, we keep:

- w_geocode: Workplace Census Block (15-digit code)

- C000: Total number of jobs lehd.ces.census.gov

- CNS05: Manufacturing jobs (NAICS 31-33) lehd.ces.census.gov

- CNS07: Retail jobs (NAICS 44-45) lehd.ces.census.gov

- CNS15: Education jobs (NAICS 61) lehd.ces.census.gov

- CNS16: Health jobs (NAICS 62) lehd.ces.census.gov

These cover the total and the example supersectors requested. (Additional sectors could be included similarly if needed.)

In [43]:
import requests
import pandas as pd

# URL for New York 2021 WAC data (All jobs, all sectors)
url_wac = "https://lehd.ces.census.gov/data/lodes/LODES8/ny/wac/ny_wac_S000_JT00_2021.csv.gz"
response = requests.get(url_wac)
open("ny_wac_2021.csv.gz", "wb").write(response.content)

# Load the WAC data for selected columns
cols = ['w_geocode', 'C000', 'CNS05', 'CNS07', 'CNS15', 'CNS16']
jobs_df = pd.read_csv("ny_wac_2021.csv.gz", usecols=cols, compression='gzip', dtype={'w_geocode': str})

# Derive block group (first 12 digits)
jobs_df['blockgroup'] = jobs_df['w_geocode'].str[:12]

# Aggregate to block group level
jobs_bg = jobs_df.groupby('blockgroup', as_index=False).sum()

print(jobs_df.head())
print("Total job records loaded:", len(jobs_df))
print("Unique blockgroups:", len(jobs_bg))


         w_geocode  C000  CNS05  CNS07  CNS15  CNS16    blockgroup
0  360010001001003     2      0      0      0      0  360010001001
1  360010001001004    51      0      0      0      0  360010001001
2  360010001001005    33      0      0      0      0  360010001001
3  360010001001006    38      0      0      0      0  360010001001
4  360010001001007   127      0      0      1      0  360010001001
Total job records loaded: 104520
Unique blockgroups: 15678


After loading, jobs_df contains one row per census block (with at least one job) in New York State, with total jobs and jobs in the specified sectors. The w_geocode is a 15-character string: we will use this to derive the block’s block group. According to Census definitions, the first 12 characters of the block code correspond to the block group (state+county+tract+block group)
catalog.data.gov
. We add a column for the 12-digit block group ID and then aggregate the employment data up to block group level:

At this point, jobs_bg has one record per block group in NY State with the total jobs and jobs by sector (summed over all blocks in that block group).

## 2. Download 2020 Census Block Group Shapefile (NYC)

Next, we retrieve the 2020 TIGER/Line shapefile for block groups and filter it to NYC. The Census FTP provides state-based shapefiles for block groups. We download the New York State block groups shapefile (tl_2020_36_bg.zip where 36 is the NY state FIPS code) and read it with GeoPandas:

In [44]:
import geopandas as gpd

# Download the NY state block group shapefile (2020)
url_bg = "https://www2.census.gov/geo/tiger/TIGER2020/BG/tl_2020_36_bg.zip"
resp = requests.get(url_bg)
open("tl_2020_36_bg.zip", "wb").write(resp.content)

# Read shapefile
bg_gdf = gpd.read_file("zip://tl_2020_36_bg.zip")
print(bg_gdf.crs, bg_gdf.columns[:5])
print("Total block groups in NY State:", len(bg_gdf))

# Filter NYC counties
nyc_counties = ["005","047","061","081","085"]
bg_nyc = bg_gdf[(bg_gdf.STATEFP == "36") & (bg_gdf.COUNTYFP.isin(nyc_counties))][['GEOID','geometry']].copy()

print("NYC block groups:", len(bg_nyc))

# Drop "0000000" (water only)
bg_nyc = bg_nyc[~bg_nyc.GEOID.str.endswith('0')].copy()
print("After removing BG ending in 0:", len(bg_nyc))


EPSG:4269 Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID'], dtype='object')
Total block groups in NY State: 16070
NYC block groups: 6807
After removing BG ending in 0: 6587


Now filter to the five NYC counties. New York City consists of Bronx, Kings, New York, Queens, and Richmond counties
en.wikipedia.org
, which have county FIPS codes 005, 047, 061, 081, and 085 respectively. We select only block groups in state FIPS 36 and those county codes:

Optional: Remove block groups with a GEOID ending in ‘0’ (block group code 0). These are special water-area-only block groups with no land
catalog.data.gov
, which should contain no jobs or population. Dropping them avoids introducing empty overlaps of water areas:

Next, we attach the jobs data to the NYC block group geometries. We left-join the aggregated jobs (jobs_bg) to the GeoDataFrame bg_nyc by matching the 12-digit GEOID:

At this stage, bg_nyc is a GeoDataFrame of NYC block groups, each with attributes: GEOID, geometry, total jobs (C000), and jobs in manufacturing (CNS05), retail (CNS07), education (CNS15), health (CNS16), plus the polygon’s area.

## 3. Load and Prepare GTFS Stations Data

Using the provided GTFS stops.txt, we load it with pandas. We filter for location_type == 1 to get only station records (GTFS defines location_type=1 as a station, while 0 is a stop/platform)
gtfs.org
. We then convert these station records into a GeoDataFrame of point geometries using the latitude and longitude:

In [45]:
import pandas as pd
import geopandas as gpd

# Load GTFS stops
stops_df = pd.read_csv("https://raw.githubusercontent.com/hawa1983/Capstone/refs/heads/main/stops.txt")

# Keep only actual stations: location_type = 1
stations_df = stops_df[stops_df['location_type'] == 1].copy()

print("Stations found:", len(stations_df))

# Convert to GeoDataFrame
stations_gdf = gpd.GeoDataFrame(
    stations_df,
    geometry=gpd.points_from_xy(stations_df.stop_lon, stations_df.stop_lat),
    crs="EPSG:4326"
)


Stations found: 499


This yields a GeoDataFrame stations_gdf with columns like stop_id, stop_name, stop_lat, stop_lon, etc., and a geometry column for the station point. According to the MTA data, there are around 499 station entries in the feed (this can include some complex station groupings or shuttles – in any case, all unique station locations will be represented).

## 4. Project to a Common CRS and Create 0.5-Mile Buffers

For spatial calculations (especially area and distance), it’s best to project the data to a planar coordinate system in meters. We use EPSG:32618 (WGS 84 / UTM Zone 18N), which covers New York City, so that distances are in meters. We convert both the station points and block group polygons to this CRS:

Now we create a 0.5-mile buffer around each station point. Half a mile is approximately 804.67 meters (since 1 mile ≈ 1609.34 m). We use GeoPandas’ buffer operation on the point geometries. To preserve the original station points for reference, we’ll make a new GeoDataFrame buffers_gdf containing station info and the buffer geometry:

In [46]:
# Job columns you need
job_cols = ['C000','CNS05','CNS07','CNS15','CNS16']

# Make sure bg_nyc is "clean" before merging (no old job columns lingering)
cols_to_drop = [c for c in job_cols + ['blockgroup'] if c in bg_nyc.columns]
if cols_to_drop:
    bg_nyc = bg_nyc.drop(columns=cols_to_drop)

# Merge jobs → block groups
bg_nyc = bg_nyc.merge(
    jobs_bg[['blockgroup'] + job_cols],
    left_on='GEOID',
    right_on='blockgroup',
    how='left'
)

# Ensure missing job columns become zero
bg_nyc[job_cols] = bg_nyc[job_cols].fillna(0)

# Project both layers BEFORE computing areas
bg_nyc = bg_nyc.to_crs(epsg=32618)
stations_gdf = stations_gdf.to_crs(epsg=32618)

# Compute area in square meters
bg_nyc['area_bg'] = bg_nyc.geometry.area


Each row in buffers_gdf is now a polygon representing the 0.5-mile radius area around a station. These are our “catchment” areas for which we want to tally jobs.

5. Spatial Join of LODES Data with Block Groups

We have two spatial layers: station buffers (polygons) and block groups (polygons with job data). To combine them, we perform a spatial overlay to identify intersections between station areas and block groups. This will allow us to calculate how much of each block group falls within each station’s buffer.

We use geopandas.overlay with how='intersection' to get polygons representing the overlap of each station-buffer and block-group pair:

In [47]:
half_mile = 0.5 * 1609.34  # meters

buffers_gdf = stations_gdf.copy()
buffers_gdf['geometry'] = stations_gdf.buffer(half_mile)

print("Buffer CRS:", buffers_gdf.crs)


Buffer CRS: EPSG:32618


The resulting intersections GeoDataFrame contains one row for each overlap area between a station buffer and a block group. It includes: the station’s stop_id (and name/coords), the block group’s GEOID, the intersecting polygon geometry, and the job counts from the block group (C000, CNS05, CNS07, etc.) that we merged earlier. If a station’s buffer covers part of a block group, that block group’s job counts are associated with the intersection polygon.

6. Area-Weighted Interpolation of Employment to Station Buffers

Now we apply area-weighted interpolation to estimate the number of jobs within each buffer. The assumption (as with population) is that jobs are uniformly distributed across the area of a block group
gis.stackexchange.com
. Thus, if (for example) 30% of a block group’s area lies inside the buffer, we allocate 30% of that block group’s jobs to the station.

Steps for each intersected polygon:

- Compute the area of the intersection (the portion of block group inside the buffer).

- Retrieve the area of the full block group (we precomputed area_bg for each block group).

- Calculate the weight = intersection_area / block_group_area (a fraction between 0 and 1).

- Multiply this weight by the block group’s job counts to get the jobs “contributed” to the station buffer from that polygon.

We perform these calculations and then sum contributions by station:

In [48]:
# Spatial overlay
intersections = gpd.overlay(bg_nyc, buffers_gdf, how='intersection')

print("Intersection features:", len(intersections))

# Compute intersected area (m²)
intersections['area_intersect'] = intersections.geometry.area

# Weight of intersected area relative to block group
intersections['weight'] = intersections['area_intersect'] / intersections['area_bg']

# Weighted job counts
intersections['jobs_total_buff']        = intersections['C000']  * intersections['weight']
intersections['jobs_retail_buff']       = intersections['CNS07'] * intersections['weight']
intersections['jobs_health_buff']       = intersections['CNS16'] * intersections['weight']
intersections['jobs_education_buff']    = intersections['CNS15'] * intersections['weight']
intersections['jobs_manufacturing_buff']= intersections['CNS05'] * intersections['weight']

# Summarize jobs per station
station_jobs = intersections.groupby('stop_id').agg(
    stop_name=('stop_name','first'),
    stop_lat=('stop_lat','first'),
    stop_lon=('stop_lon','first'),
    total_jobs=('jobs_total_buff','sum'),
    jobs_retail=('jobs_retail_buff','sum'),
    jobs_health=('jobs_health_buff','sum'),
    jobs_education=('jobs_education_buff','sum'),
    jobs_manufacturing=('jobs_manufacturing_buff','sum')
).reset_index()


Intersection features: 22283


After grouping, station_jobs contains one row per station with the aggregated job counts within 0.5 miles. The columns include the station’s ID, name, original lat/lon, total jobs, and jobs in retail, health, education, and manufacturing sectors within the buffer.

**Note on assumptions:** This area-weighting method assumes uniform job distribution in each block group
gis.stackexchange.com
. In reality, jobs may cluster (e.g., along commercial corridors), so there is some interpolation error. A more refined approach could use smaller spatial units (census blocks or a dasymetric method)
gis.stackexchange.com
gis.stackexchange.com
, but the area-weighted block group approach is a reasonable approximation given available data, and it matches the population methodology.

## 7. Output the Results

Finally, we save the results to a CSV as specified:

In [49]:
pd.set_option('display.float_format', lambda x: f'{x:,.0f}')

print(
    station_jobs[[
        'stop_id',
        'stop_name',
        'stop_lat',
        'stop_lon',
        'total_jobs',
        'jobs_retail',
        'jobs_health',
        'jobs_education',
        'jobs_manufacturing'
    ]].head(10)
)

station_jobs.to_csv("station_jobs_0.5mile.csv", index=False)


  stop_id                  stop_name  stop_lat  stop_lon  total_jobs  \
0     101  Van Cortlandt Park-242 St        41       -74       5,323   
1     103                     238 St        41       -74       7,046   
2     104                     231 St        41       -74       7,101   
3     106         Marble Hill-225 St        41       -74       6,262   
4     107                     215 St        41       -74       5,624   
5     108                     207 St        41       -74       5,859   
6     109                 Dyckman St        41       -74       6,529   
7     110                     191 St        41       -74      10,919   
8     111                     181 St        41       -74      13,736   
9     112      168 St-Washington Hts        41       -74      20,906   

   jobs_retail  jobs_health  jobs_education  jobs_manufacturing  
0          637          685           2,356                   5  
1        1,119        1,401           1,661                   4  
2        

Each row of station_jobs_0.5mile.csv contains:

- stop_id – Station ID from GTFS

- stop_name – Station name

- lat, lon – Station latitude and longitude (WGS84)

- total_jobs – Total jobs within 0.5 miles of the station

- jobs_retail – Retail sector jobs (NAICS 44-45) within 0.5 miles

- jobs_health – Health care and social assistance jobs (NAICS 62)

- jobs_education – Educational services jobs (NAICS 61)

- jobs_manufacturing – Manufacturing jobs (NAICS 31-33)

And similarly for any other sectors if included. These figures are the sum of area-weighted contributions from all block groups that intersect the station’s half-mile buffer.

The output CSV can be used for analysis of job accessibility around transit. For example, one could rank stations by total jobs accessible within walking distance, or examine which stations have high concentrations of retail vs. healthcare jobs, etc. The approach is general and can be adapted to other geographies or additional NAICS categories as needed, using open data and Python GIS libraries.

**Sources:** Data from U.S. Census LEHD LODES
census.gov
lehd.ces.census.gov
 and TIGER/Line 2020; MTA GTFS stops
gtfs.org
. The methodology follows standard areal interpolation practices
gis.stackexchange.com
.

# Vehicle Availability near NYC Subway Stations (ACS 2020 Analysis)
## 1. Retrieve ACS 2020 Data for Household Vehicle Availability (B08201)

We first obtain American Community Survey (ACS) 5-Year 2020 estimates for table B08201: Household Size by Vehicles Available at the block group level. In particular, we need two variables from this table:

- B08201_001E – Total households api.census.gov

- B08201_002E – Households with no vehicle available api.census.gov

These correspond to the total number of households and the subset with zero vehicles, respectively (the table also includes households with 1, 2, 3, 4+ vehicles)
censusreporter.org
. We can use the CenPy or censusdata library to query the Census API for these variables at the block group geography in New York State. For example, using the censusdata library:

This will download the total households and zero-vehicle households for every block group in New York State. If using CenPy, one could similarly specify the product and table:

# Vehicle Availability near NYC Subway Stations (ACS 2020 Analysis)

## Pull ACS 2020 B08201 at block group level (NY)

Add this once (analog of your LODES download + aggregation):

In [50]:
import requests
import pandas as pd

# ACS 2020 5-year detailed tables
acs_base_url = "https://api.census.gov/data/2020/acs/acs5"

acs_vars = ["B08201_001E", "B08201_002E"]  # total HH, HH with no vehicle

acs_params = {
    "get": ",".join(["NAME"] + acs_vars),
    "for": "block group:*",
    "in": "state:36+county:*",  # all NY state block groups
    # "key": "YOUR_CENSUS_API_KEY",  # optional but recommended
}

resp = requests.get(acs_base_url, params=acs_params)
resp.raise_for_status()

data = resp.json()
acs_bg = pd.DataFrame(data[1:], columns=data[0])

# Construct 12-digit block group GEOID
acs_bg["GEOID"] = (
    acs_bg["state"] + acs_bg["county"] + acs_bg["tract"] + acs_bg["block group"]
)

# Cast numeric fields
acs_bg[acs_vars] = acs_bg[acs_vars].astype(float)


In [55]:
acs_bg[["GEOID"]].head()
acs_bg.shape
acs_bg[["B08201_001E", "B08201_002E"]].describe()
acs_bg[acs_bg["GEOID"] == "360610239001"]


,NAME,B08201_001E,B08201_002E,state,county,tract,block group,GEOID
15763,"Block Group 1, Census Tract 239, New York Coun...",NaN,NaN,36,061,023900,1,360610239001


## Merge ACS variables into bg_nyc

Right after you create bg_nyc from the TIGER shapefile (and filter to NYC counties, drop water BGs), but before projection / area calculations, add:

In [54]:
# Drop old ACS columns if re-running
for c in ["B08201_001E", "B08201_002E"]:
    if c in bg_nyc.columns:
        bg_nyc = bg_nyc.drop(columns=[c])

# Merge ACS onto NYC BGs by GEOID
bg_nyc = bg_nyc.merge(
    acs_bg[["GEOID", "B08201_001E", "B08201_002E"]],
    on="GEOID",
    how="left"
)

# Replace missing with 0 just in case
bg_nyc[["B08201_001E", "B08201_002E"]] = (
    bg_nyc[["B08201_001E", "B08201_002E"]].fillna(0)
)

bg_nyc = bg_nyc.to_crs(epsg=32618)
bg_nyc["area_bg"] = bg_nyc.geometry.area

bg_nyc[["B08201_001E", "B08201_002E"]].describe()
bg_nyc[["GEOID", "B08201_001E", "B08201_002E"]].head()


,GEOID,B08201_001E,B08201_002E
0,360610239001,0,0
1,360610139001,0,0
2,360610078002,0,0
3,360610089001,0,0
4,360610089004,0,0


## Area-weighted households in the intersection step

Where you previously did jobs area-weighting on intersections, add a separate block (or a separate notebook cell) for households:

In [52]:
import geopandas as gpd

# Reuse your buffer GeoDataFrame: buffers_gdf
# Reuse your overlay pattern:
intersections_hh = gpd.overlay(bg_nyc, buffers_gdf, how="intersection")

# Areas
intersections_hh["area_intersect"] = intersections_hh.geometry.area
intersections_hh["weight"] = (
    intersections_hh["area_intersect"] / intersections_hh["area_bg"]
)

# Area-weighted estimates
intersections_hh["hh_total_est"] = (
    intersections_hh["B08201_001E"] * intersections_hh["weight"]
)
intersections_hh["hh_no_vehicle_est"] = (
    intersections_hh["B08201_002E"] * intersections_hh["weight"]
)


## Aggregate to station and compute vehicle percentages

Analog of your station_jobs groupby:

In [53]:
station_veh = intersections_hh.groupby("stop_id").agg(
    stop_name         = ("stop_name", "first"),
    stop_lat          = ("stop_lat", "first"),
    stop_lon          = ("stop_lon", "first"),
    hh_total_est      = ("hh_total_est", "sum"),
    hh_no_vehicle_est = ("hh_no_vehicle_est", "sum"),
).reset_index()

station_veh["hh_with_vehicle_est"] = (
    station_veh["hh_total_est"] - station_veh["hh_no_vehicle_est"]
)

station_veh["pct_no_vehicle"] = (
    station_veh["hh_no_vehicle_est"] / station_veh["hh_total_est"] * 100
)
station_veh["pct_with_vehicle"] = 100 - station_veh["pct_no_vehicle"]

station_veh["pct_no_vehicle"] = station_veh["pct_no_vehicle"].round(1)
station_veh["pct_with_vehicle"] = station_veh["pct_with_vehicle"].round(1)

output_veh = station_veh.rename(columns={
    "hh_total_est": "households_total",
    "stop_lat": "lat",
    "stop_lon": "lon",
})[[
    "stop_id", "stop_name", "lat", "lon",
    "households_total", "pct_no_vehicle", "pct_with_vehicle",
]]

print(output_veh.head())
output_veh.to_csv("nyc_subway_vehicle_availability_0p5mile.csv", index=False)


  stop_id                  stop_name  lat  lon  households_total  \
0     101  Van Cortlandt Park-242 St   41  -74                 0   
1     103                     238 St   41  -74                 0   
2     104                     231 St   41  -74                 0   
3     106         Marble Hill-225 St   41  -74                 0   
4     107                     215 St   41  -74                 0   

   pct_no_vehicle  pct_with_vehicle  
0             NaN               NaN  
1             NaN               NaN  
2             NaN               NaN  
3             NaN               NaN  
4             NaN               NaN  


In [60]:
import requests
import pandas as pd
import geopandas as gpd

# ============================================================
# 0. CONFIG
# ============================================================
# Your Census API key
API_KEY = "a4373ffe644694a60eeb7c0bae0bedcfd2d6ff78"

projected_crs = "EPSG:32618"     # UTM Zone 18N (meters)
half_mile_m = 0.5 * 1609.34      # 0.5 mile in meters

# NYC county FIPS (same as your population / jobs code)
nyc_county_fips = ["005", "047", "061", "081", "085"]

# ============================================================
# 1. LOAD SUBWAY STATIONS (GTFS stops.txt)
#    (you can swap in your existing loading code if you prefer)
# ============================================================
stops_url = "https://raw.githubusercontent.com/hawa1983/Capstone/refs/heads/main/stops.txt"
stops_df = pd.read_csv(stops_url)

# Keep only station-level entries: location_type == 1
stops_df["location_type"] = stops_df["location_type"].fillna(0).astype(int)
stations_df = stops_df[stops_df["location_type"] == 1].copy()

print("Stations found:", len(stations_df))

# GeoDataFrame in WGS84
stations_gdf = gpd.GeoDataFrame(
    stations_df,
    geometry=gpd.points_from_xy(stations_df.stop_lon, stations_df.stop_lat),
    crs="EPSG:4326"
)

# Preserve original lat/lon for output (so projection doesn't affect them)
stations_gdf["orig_lat"] = stations_gdf.geometry.y
stations_gdf["orig_lon"] = stations_gdf.geometry.x

# ============================================================
# 2. LOAD 2020 BLOCK GROUP SHAPEFILE (NYC)
# ============================================================
shapefile_url = "https://www2.census.gov/geo/tiger/TIGER2020/BG/tl_2020_36_bg.zip"
bg_gdf = gpd.read_file(shapefile_url)

print("Total block groups in NY State:", len(bg_gdf))

# Filter to NYC counties (Bronx, Kings, New York, Queens, Richmond)
bg_nyc = bg_gdf[bg_gdf["COUNTYFP"].isin(nyc_county_fips)].copy()
print("NYC block groups (before water drop):", len(bg_nyc))

# Drop water-only block groups: GEOID ending in "0"
bg_nyc = bg_nyc[~bg_nyc["GEOID"].str.endswith("0")].copy()
print("NYC block groups (after dropping BG 0):", len(bg_nyc))

# ============================================================
# 3. RETRIEVE ACS 2020 B08201 FOR NYC BLOCK GROUPS
#    (per-county loop, like your population code)
# ============================================================
acs_list = []
for county in nyc_county_fips:
    url = (
        "https://api.census.gov/data/2020/acs/acs5"
        "?get=B08201_001E,B08201_002E"
        f"&for=block%20group:*&in=state:36&in=county:{county}"
        f"&key={API_KEY}"
    )
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()
    cols = data[0]
    rows = data[1:]
    df = pd.DataFrame(rows, columns=cols)
    acs_list.append(df)

acs_df = pd.concat(acs_list, ignore_index=True)

# Rename and cast
acs_df.rename(columns={
    "B08201_001E": "hh_total",
    "B08201_002E": "hh_zero_vehicle"
}, inplace=True)

acs_df["hh_total"] = pd.to_numeric(acs_df["hh_total"], errors="coerce")
acs_df["hh_zero_vehicle"] = pd.to_numeric(acs_df["hh_zero_vehicle"], errors="coerce")

# Construct 12-digit block group GEOID
acs_df["GEOID"] = (
    acs_df["state"] + acs_df["county"] + acs_df["tract"] + acs_df["block group"]
)

print("ACS rows:", len(acs_df))
print(acs_df.head(3))

# ============================================================
# 4. MERGE ACS VEHICLE DATA INTO NYC BLOCK GROUP GEOMETRIES
# ============================================================
bg_nyc["GEOID"] = bg_nyc["GEOID"].astype(str)

bg_nyc = bg_nyc.merge(
    acs_df[["GEOID", "hh_total", "hh_zero_vehicle"]],
    on="GEOID",
    how="left"
)

# Replace missing with 0 (should be rare)
bg_nyc[["hh_total", "hh_zero_vehicle"]] = bg_nyc[["hh_total", "hh_zero_vehicle"]].fillna(0)

print("After merge, hh_total summary:")
print(bg_nyc["hh_total"].describe())

# ============================================================
# 5. PROJECT TO PLANAR CRS AND CREATE 0.5-MILE BUFFERS
# ============================================================
bg_nyc_proj = bg_nyc.to_crs(projected_crs)
stations_proj = stations_gdf.to_crs(projected_crs)

# Block group area (m²)
bg_nyc_proj["area_bg"] = bg_nyc_proj.geometry.area

# Station buffers
buffers_gdf = stations_proj.copy()
buffers_gdf["geometry"] = stations_proj.buffer(half_mile_m)

print("Buffer CRS:", buffers_gdf.crs)

# ============================================================
# 6. SPATIAL OVERLAY + AREA-WEIGHTED HOUSEHOLDS
#    (like your jobs overlay)
# ============================================================
intersections = gpd.overlay(bg_nyc_proj, buffers_gdf, how="intersection")
print("Intersection features:", len(intersections))

# Area of intersection and weight
intersections["area_intersect"] = intersections.geometry.area
intersections["weight"] = intersections["area_intersect"] / intersections["area_bg"]

# Weighted household estimates
intersections["hh_total_est"] = intersections["hh_total"] * intersections["weight"]
intersections["hh_zero_vehicle_est"] = intersections["hh_zero_vehicle"] * intersections["weight"]

# ============================================================
# 7. AGGREGATE BY STATION AND COMPUTE PERCENTAGES
# ============================================================
station_veh = intersections.groupby("stop_id").agg(
    stop_name=("stop_name", "first"),
    # use original lat/lon from stations_gdf via buffers_gdf
    lat=("orig_lat", "first"),
    lon=("orig_lon", "first"),
    hh_total_est=("hh_total_est", "sum"),
    hh_zero_vehicle_est=("hh_zero_vehicle_est", "sum")
).reset_index()

# Drop stations with zero estimated households to avoid 0/0
station_veh = station_veh[station_veh["hh_total_est"] > 0].copy()

station_veh["hh_with_vehicle_est"] = (
    station_veh["hh_total_est"] - station_veh["hh_zero_vehicle_est"]
)

station_veh["pct_no_vehicle"] = (
    station_veh["hh_zero_vehicle_est"] / station_veh["hh_total_est"] * 100
)
station_veh["pct_with_vehicle"] = 100 - station_veh["pct_no_vehicle"]

station_veh["pct_no_vehicle"] = station_veh["pct_no_vehicle"].round(1)
station_veh["pct_with_vehicle"] = station_veh["pct_with_vehicle"].round(1)

# ============================================================
# 8. FINAL OUTPUT
# ============================================================
output_veh = station_veh.rename(columns={
    "hh_total_est": "households_total"
})[[
    "stop_id",
    "stop_name",
    "lat",
    "lon",
    "households_total",
    "pct_no_vehicle",
    "pct_with_vehicle"
]]

print(output_veh.head(10))

output_veh.to_csv("nyc_subway_vehicle_availability_0p5mile.csv", index=False)
print("Saved nyc_subway_vehicle_availability_0p5mile.csv")


Stations found: 499
Total block groups in NY State: 16070
NYC block groups (before water drop): 6807
NYC block groups (after dropping BG 0): 6587
ACS rows: 6807
   hh_total  hh_zero_vehicle state county   tract block group         GEOID
0       NaN              NaN    36    005  000100           1  360050001001
1       NaN              NaN    36    005  000200           2  360050002002
2       NaN              NaN    36    005  002001           1  360050020011
After merge, hh_total summary:
count   6,587
mean        0
std         0
min         0
25%         0
50%         0
75%         0
max         0
Name: hh_total, dtype: float64
Buffer CRS: EPSG:32618
Intersection features: 22283
Empty DataFrame
Columns: [stop_id, stop_name, lat, lon, households_total, pct_no_vehicle, pct_with_vehicle]
Index: []
Saved nyc_subway_vehicle_availability_0p5mile.csv


In [61]:
import requests
import pandas as pd
import geopandas as gpd

# ============================================================
# 0. CONFIG
# ============================================================
# Your Census API key (same one you used for population)
API_KEY = "a4373ffe644694a60eeb7c0bae0bedcfd2d6ff78"

# Projected CRS and buffer distance
PROJECTED_CRS = "EPSG:32618"          # UTM Zone 18N (meters)
HALF_MILE_M   = 0.5 * 1609.34         # 0.5 mile in meters

# NYC county FIPS codes (Bronx, Kings, New York, Queens, Richmond)
NYC_COUNTIES = ["005", "047", "061", "081", "085"]


# ============================================================
# 1. LOAD SUBWAY STATIONS (GTFS stops.txt)
#    (same style as your jobs script)
# ============================================================
stops_url = "https://raw.githubusercontent.com/hawa1983/Capstone/refs/heads/main/stops.txt"
stops_df = pd.read_csv(stops_url)

# Ensure location_type is usable as int
stops_df["location_type"] = stops_df["location_type"].fillna(0).astype(int)

# Keep only station-level entries: location_type == 1
stations_df = stops_df[stops_df["location_type"] == 1].copy()
print("Stations found:", len(stations_df))

# GeoDataFrame in WGS84
stations_gdf = gpd.GeoDataFrame(
    stations_df,
    geometry=gpd.points_from_xy(stations_df.stop_lon, stations_df.stop_lat),
    crs="EPSG:4326"
)

# Preserve original lat/lon for output
stations_gdf["orig_lat"] = stations_gdf.geometry.y
stations_gdf["orig_lon"] = stations_gdf.geometry.x


# ============================================================
# 2. LOAD 2020 BLOCK GROUP SHAPEFILE (NYC)
#    (same idea as your population + jobs code)
# ============================================================
shapefile_url = "https://www2.census.gov/geo/tiger/TIGER2020/BG/tl_2020_36_bg.zip"
block_groups_gdf = gpd.read_file(shapefile_url)
print("Total block groups in NY State:", len(block_groups_gdf))

# Filter to NYC’s five counties
bg_nyc = block_groups_gdf[block_groups_gdf["COUNTYFP"].isin(NYC_COUNTIES)].copy()
print("NYC block groups (before dropping water BG 0):", len(bg_nyc))

# Drop "BG 0" (water-only) using GEOID ending in '0'
bg_nyc = bg_nyc[~bg_nyc["GEOID"].str.endswith("0")].copy()
print("NYC block groups (after dropping BG 0):", len(bg_nyc))


# ============================================================
# 3. RETRIEVE ACS 2020 B08201 FOR NYC BLOCK GROUPS
#    (loop over counties, like your population script)
# ============================================================
acs_list = []

for county in NYC_COUNTIES:
    url = (
        "https://api.census.gov/data/2020/acs/acs5"
        "?get=B08201_001E,B08201_002E"
        f"&for=block%20group:*&in=state:36&in=county:{county}"
        f"&key={API_KEY}"
    )
    resp = requests.get(url)
    resp.raise_for_status()
    data = resp.json()
    cols = data[0]
    rows = data[1:]
    df = pd.DataFrame(rows, columns=cols)
    acs_list.append(df)

acs_df = pd.concat(acs_list, ignore_index=True)

# Rename and convert to numeric
acs_df.rename(columns={
    "B08201_001E": "hh_total",
    "B08201_002E": "hh_zero_vehicle"
}, inplace=True)

acs_df["hh_total"] = pd.to_numeric(acs_df["hh_total"], errors="coerce")
acs_df["hh_zero_vehicle"] = pd.to_numeric(acs_df["hh_zero_vehicle"], errors="coerce")

# Construct 12-digit GEOID for block group
acs_df["GEOID"] = (
    acs_df["state"] +
    acs_df["county"] +
    acs_df["tract"] +
    acs_df["block group"]
)

print("ACS rows:", len(acs_df))
print(acs_df.head(3))


# ============================================================
# 4. MERGE ACS VEHICLE DATA INTO BLOCK GROUP GEOMETRIES
# ============================================================
bg_nyc["GEOID"] = bg_nyc["GEOID"].astype(str)

bg_nyc = bg_nyc.merge(
    acs_df[["GEOID", "hh_total", "hh_zero_vehicle"]],
    on="GEOID",
    how="left"
)

# Replace missing with 0 (should be rare)
bg_nyc[["hh_total", "hh_zero_vehicle"]] = bg_nyc[["hh_total", "hh_zero_vehicle"]].fillna(0)

print("After merge, hh_total summary:")
print(bg_nyc["hh_total"].describe())


# ============================================================
# 5. PROJECT TO PLANAR CRS AND CREATE 0.5-MILE BUFFERS
#    (same projection + buffer pattern as population/jobs)
# ============================================================
bg_nyc_proj = bg_nyc.to_crs(PROJECTED_CRS)
stations_proj = stations_gdf.to_crs(PROJECTED_CRS)

# Block group area in square meters
bg_nyc_proj["area_bg"] = bg_nyc_proj.geometry.area

# Create buffers around each station
buffers_gdf = stations_proj.copy()
buffers_gdf["geometry"] = stations_proj.buffer(HALF_MILE_M)

print("Buffer CRS:", buffers_gdf.crs)


# ============================================================
# 6. SPATIAL OVERLAY + AREA-WEIGHTED HOUSEHOLDS
#    (same style as your jobs overlay)
# ============================================================
intersections = gpd.overlay(bg_nyc_proj, buffers_gdf, how="intersection")
print("Intersection features:", len(intersections))

# Intersection area and area weight
intersections["area_intersect"] = intersections.geometry.area
intersections["weight"] = intersections["area_intersect"] / intersections["area_bg"]

# Area-weighted household estimates
intersections["hh_total_est"] = intersections["hh_total"] * intersections["weight"]
intersections["hh_zero_vehicle_est"] = intersections["hh_zero_vehicle"] * intersections["weight"]


# ============================================================
# 7. AGGREGATE BY STATION AND COMPUTE PERCENTAGES
# ============================================================
station_veh = intersections.groupby("stop_id").agg(
    stop_name=("stop_name", "first"),
    lat=("orig_lat", "first"),
    lon=("orig_lon", "first"),
    hh_total_est=("hh_total_est", "sum"),
    hh_zero_vehicle_est=("hh_zero_vehicle_est", "sum")
).reset_index()

# Drop stations with zero estimated households to avoid 0/0
station_veh = station_veh[station_veh["hh_total_est"] > 0].copy()

station_veh["hh_with_vehicle_est"] = (
    station_veh["hh_total_est"] - station_veh["hh_zero_vehicle_est"]
)

station_veh["pct_no_vehicle"] = (
    station_veh["hh_zero_vehicle_est"] / station_veh["hh_total_est"] * 100
)
station_veh["pct_with_vehicle"] = 100 - station_veh["pct_no_vehicle"]

station_veh["pct_no_vehicle"] = station_veh["pct_no_vehicle"].round(1)
station_veh["pct_with_vehicle"] = station_veh["pct_with_vehicle"].round(1)


# ============================================================
# 8. FINAL OUTPUT
# ============================================================
output_veh = station_veh.rename(columns={
    "hh_total_est": "households_total"
})[[
    "stop_id",
    "stop_name",
    "lat",
    "lon",
    "households_total",
    "pct_no_vehicle",
    "pct_with_vehicle"
]]

print(output_veh.head(10))

output_veh.to_csv("nyc_subway_vehicle_availability_0p5mile.csv", index=False)
print("Saved nyc_subway_vehicle_availability_0p5mile.csv")


Stations found: 499
Total block groups in NY State: 16070
NYC block groups (before dropping water BG 0): 6807
NYC block groups (after dropping BG 0): 6587
ACS rows: 6807
   hh_total  hh_zero_vehicle state county   tract block group         GEOID
0       NaN              NaN    36    005  000100           1  360050001001
1       NaN              NaN    36    005  000200           2  360050002002
2       NaN              NaN    36    005  002001           1  360050020011
After merge, hh_total summary:
count   6,587
mean        0
std         0
min         0
25%         0
50%         0
75%         0
max         0
Name: hh_total, dtype: float64
Buffer CRS: EPSG:32618
Intersection features: 22283
Empty DataFrame
Columns: [stop_id, stop_name, lat, lon, households_total, pct_no_vehicle, pct_with_vehicle]
Index: []
Saved nyc_subway_vehicle_availability_0p5mile.csv


In [62]:
import requests
import pandas as pd
import geopandas as gpd

# ============================================================
# 0. CONFIG
# ============================================================
API_KEY = "a4373ffe644694a60eeb7c0bae0bedcfd2d6ff78"  # your key

projected_crs = "EPSG:32618"     # UTM Zone 18N (meters)
half_mile_m = 0.5 * 1609.34      # 0.5 mile in meters

# NYC county FIPS
nyc_county_fips = ["005", "047", "061", "081", "085"]


# ============================================================
# 1. LOAD SUBWAY STATIONS (GTFS stops.txt)
# ============================================================
stops_url = "https://raw.githubusercontent.com/hawa1983/Capstone/refs/heads/main/stops.txt"
stops_df = pd.read_csv(stops_url)

# Keep only station-level entries: location_type == 1
stops_df["location_type"] = stops_df["location_type"].fillna(0).astype(int)
stations_df = stops_df[stops_df["location_type"] == 1].copy()

print("Stations found:", len(stations_df))

# GeoDataFrame in WGS84
stations_gdf = gpd.GeoDataFrame(
    stations_df,
    geometry=gpd.points_from_xy(stations_df.stop_lon, stations_df.stop_lat),
    crs="EPSG:4326"
)

# Preserve original lat/lon for output
stations_gdf["orig_lat"] = stations_gdf.geometry.y
stations_gdf["orig_lon"] = stations_gdf.geometry.x


# ============================================================
# 2. LOAD 2020 BLOCK GROUP SHAPEFILE (NYC)
# ============================================================
shapefile_url = "https://www2.census.gov/geo/tiger/TIGER2020/BG/tl_2020_36_bg.zip"
bg_gdf = gpd.read_file(shapefile_url)

print("Total block groups in NY State:", len(bg_gdf))

# Filter to NYC counties (Bronx, Kings, New York, Queens, Richmond)
bg_nyc = bg_gdf[bg_gdf["COUNTYFP"].isin(nyc_county_fips)].copy()
print("NYC block groups (before water drop):", len(bg_nyc))

# Drop water-only block groups: GEOID ending in "0"
bg_nyc = bg_nyc[~bg_nyc["GEOID"].str.endswith("0")].copy()
print("NYC block groups (after dropping BG 0):", len(bg_nyc))


# ============================================================
# 3. RETRIEVE **2019** ACS B08201 FOR NYC BLOCK GROUPS
# ============================================================
acs_list = []
for county in nyc_county_fips:
    url = (
        "https://api.census.gov/data/2019/acs/acs5"
        "?get=B08201_001E,B08201_002E"
        f"&for=block%20group:*&in=state:36&in=county:{county}"
        f"&key={API_KEY}"
    )
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()
    cols = data[0]
    rows = data[1:]
    df = pd.DataFrame(rows, columns=cols)
    acs_list.append(df)

acs_df = pd.concat(acs_list, ignore_index=True)

# Rename and cast
acs_df.rename(columns={
    "B08201_001E": "hh_total",
    "B08201_002E": "hh_zero_vehicle"
}, inplace=True)

acs_df["hh_total"] = pd.to_numeric(acs_df["hh_total"], errors="coerce")
acs_df["hh_zero_vehicle"] = pd.to_numeric(acs_df["hh_zero_vehicle"], errors="coerce")

# Construct 12-digit block group GEOID
acs_df["GEOID"] = (
    acs_df["state"] + acs_df["county"] + acs_df["tract"] + acs_df["block group"]
)

print("ACS rows:", len(acs_df))
print(acs_df.head(3))


# ============================================================
# 4. MERGE ACS VEHICLE DATA INTO NYC BLOCK GROUP GEOMETRIES
# ============================================================
bg_nyc["GEOID"] = bg_nyc["GEOID"].astype(str)

bg_nyc = bg_nyc.merge(
    acs_df[["GEOID", "hh_total", "hh_zero_vehicle"]],
    on="GEOID",
    how="left"
)

# Replace missing with 0 (should be rare)
bg_nyc[["hh_total", "hh_zero_vehicle"]] = bg_nyc[["hh_total", "hh_zero_vehicle"]].fillna(0)

print("After merge, hh_total summary:")
print(bg_nyc["hh_total"].describe())
print("\nSample merged rows:")
print(bg_nyc[["GEOID", "hh_total", "hh_zero_vehicle"]].head())


# ============================================================
# 5. PROJECT TO PLANAR CRS AND CREATE 0.5-MILE BUFFERS
# ============================================================
bg_nyc_proj = bg_nyc.to_crs(projected_crs)
stations_proj = stations_gdf.to_crs(projected_crs)

# Block group area (m²)
bg_nyc_proj["area_bg"] = bg_nyc_proj.geometry.area

# Station buffers
buffers_gdf = stations_proj.copy()
buffers_gdf["geometry"] = stations_proj.buffer(half_mile_m)

print("Buffer CRS:", buffers_gdf.crs)


# ============================================================
# 6. SPATIAL OVERLAY + AREA-WEIGHTED HOUSEHOLDS
# ============================================================
intersections = gpd.overlay(bg_nyc_proj, buffers_gdf, how="intersection")
print("Intersection features:", len(intersections))

# Area of intersection and weight
intersections["area_intersect"] = intersections.geometry.area
intersections["weight"] = intersections["area_intersect"] / intersections["area_bg"]

# Weighted household estimates
intersections["hh_total_est"] = intersections["hh_total"] * intersections["weight"]
intersections["hh_zero_vehicle_est"] = intersections["hh_zero_vehicle"] * intersections["weight"]


# ============================================================
# 7. AGGREGATE BY STATION AND COMPUTE PERCENTAGES
# ============================================================
station_veh = intersections.groupby("stop_id").agg(
    stop_name=("stop_name", "first"),
    lat=("orig_lat", "first"),
    lon=("orig_lon", "first"),
    hh_total_est=("hh_total_est", "sum"),
    hh_zero_vehicle_est=("hh_zero_vehicle_est", "sum")
).reset_index()

# Drop stations with zero estimated households to avoid 0/0
station_veh = station_veh[station_veh["hh_total_est"] > 0].copy()

station_veh["hh_with_vehicle_est"] = (
    station_veh["hh_total_est"] - station_veh["hh_zero_vehicle_est"]
)

station_veh["pct_no_vehicle"] = (
    station_veh["hh_zero_vehicle_est"] / station_veh["hh_total_est"] * 100
)
station_veh["pct_with_vehicle"] = 100 - station_veh["pct_no_vehicle"]

station_veh["pct_no_vehicle"] = station_veh["pct_no_vehicle"].round(1)
station_veh["pct_with_vehicle"] = station_veh["pct_with_vehicle"].round(1)


# ============================================================
# 8. FINAL OUTPUT
# ============================================================
output_veh = station_veh.rename(columns={
    "hh_total_est": "households_total"
})[[
    "stop_id",
    "stop_name",
    "lat",
    "lon",
    "households_total",
    "pct_no_vehicle",
    "pct_with_vehicle"
]]

print(output_veh.head(10))

output_veh.to_csv("nyc_subway_vehicle_availability_0p5mile.csv", index=False)
print("Saved nyc_subway_vehicle_availability_0p5mile.csv")


Stations found: 499
Total block groups in NY State: 16070
NYC block groups (before water drop): 6807
NYC block groups (after dropping BG 0): 6587
ACS rows: 6493
   hh_total  hh_zero_vehicle state county   tract block group         GEOID
0       NaN              NaN    36    005  046000           1  360050460001
1       NaN              NaN    36    005  046000           2  360050460002
2       NaN              NaN    36    005  039800           3  360050398003
After merge, hh_total summary:
count   6,587
mean        0
std         0
min         0
25%         0
50%         0
75%         0
max         0
Name: hh_total, dtype: float64

Sample merged rows:
          GEOID  hh_total  hh_zero_vehicle
0  360610239001         0                0
1  360610139001         0                0
2  360610078002         0                0
3  360610089001         0                0
4  360610089004         0                0
Buffer CRS: EPSG:32618
Intersection features: 22283
Empty DataFrame
Columns: [stop